<a href="https://colab.research.google.com/github/tinaghimire/face_similarity/blob/main/Face_segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image Segmentation

## Variants

### Semantic Segmentation

- Assigns probabilities of classes to every pixel.
- One segment per class like red border around all cat in an image

### Instance Segmentation

- Segments every distinct object
- Different segment for each object like different color for every cat in an image

### Panoptic Segmentation

- Segment the image both instance-wise and class-wise

## Metrics

- Intersection-over-union (IOU)
- Average Precision
- Mean Average Precision (mAP)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Dataset Processing

In [2]:
!pip install torch torchvision

In [15]:
import cv2
import numpy as np
import os
import torch
import torchvision

def combine_masks_to_yolo(img_dir, mask_dir, save_dir, classes):

    for image in os.listdir(img_dir):
        img = cv2.imread(os.path.join(img_dir, image))
        image_prefix = image.split('.')[0]

        height, width, _ = img.shape

        mask_annotate = []
        for mask in os.listdir(mask_dir):
            if mask.startswith(f"0000{image_prefix}"):
                mask_class = mask.split('_',1)[1].split('.')[0]
                mask_img = cv2.imread(os.path.join(mask_dir, mask), cv2.IMREAD_GRAYSCALE)

                contours, _ = cv2.findContours((mask_img > 0).astype(np.uint8), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                class_id = classes[mask_class]

                for contour in contours:
                    if cv2.contourArea(contour) > 100:
                        polygon = []
                        for point in contour:
                            x, y = point[0]
                            polygon.append(x / width)
                            polygon.append(y / height)

                        text = f"{class_id} " + " ".join(map(str, polygon))
                        mask_annotate.append(text)

        if mask_annotate:
            with open(os.path.join(save_dir, f"{image_prefix}.txt"), 'w') as f:
                f.write('\n'.join(mask_annotate))

In [17]:
classes = {
    'eyes': 0,
    'ear': 1,
    'nose': 2,
    'mouth': 3,
    'brow': 4,
    'neck':5
}

img_dir = '/content/drive/MyDrive/Face_similarity/Dataset/images'
mask_dir = '/content/drive/MyDrive/Face_similarity/Dataset/masks'
save_dir = '/content/drive/MyDrive/Face_similarity/Dataset/labels'

combine_masks_to_yolo(img_dir, mask_dir, save_dir, classes)

In [30]:
from sklearn.model_selection import train_test_split
import pandas as pd
import shutil

image_data = []
labels_data = []

for img, labels in zip(os.listdir(img_dir), os.listdir(save_dir)):
    img_path = os.path.join(img_dir, img)
    labels_path = os.path.join(save_dir, labels)

    image_data.append(img_path)
    labels_data.append(labels_path)

train_images, val_images, train_labels, val_labels = train_test_split(image_data, labels_data, test_size=0.2, random_state=42)
print(len(train_images), len(val_images), len(train_labels), len(val_labels))
print(train_images)

80 20 80 20
['/content/drive/MyDrive/Face_similarity/Dataset/images/43.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/91.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/00.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/82.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/59.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/18.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/47.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/69.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/22.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/44.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/25.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/86.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/65.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/34.jpg', '/content/drive/MyDrive/Face_similarity/Dataset/images/68.jpg', '/content/drive/MyDrive/Fac

In [33]:
# for images in train_images:
#     print(images)
#     shutil.copy(images, '/content/drive/MyDrive/Face_similarity/Dataset/images/train/')

# for images in val_images:
#     shutil.copy(images, '/content/drive/MyDrive/Face_similarity/Dataset/images/val/')

# for labels in train_labels:
#     shutil.copy(labels, '/content/drive/MyDrive/Face_similarity/Dataset/labels/train/')

# for labels in val_labels:
#     shutil.copy(labels, '/content/drive/MyDrive/Face_similarity/Dataset/labels/val/')

## Training

In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.0/887.0 kB 42.7 MB/s eta 0:00:00


In [6]:
config = '''
    path: /content/drive/MyDrive/Face_similarity/Dataset
    train: images/train
    val: images/val
    nc: 6
    names:
        0: eyes
        1: ear
        2: nose
        3: mouth
        4: brow
        5: neck
'''

with open('config.yaml', 'w') as f:
    f.write(config)

In [7]:
from ultralytics import YOLO

model = YOLO('yolov8n-seg.pt')

model.train(data="config.yaml", epochs=100, imgsz=640)

Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=config.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, l

100%|██████████| 755k/755k [00:00<00:00, 128MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 266MB/s]


AMP: checks passed ✅


...

                   all         20        158      0.925      0.952      0.947      0.684      0.925      0.942      0.935      0.623



100 epochs completed in 0.138 hours.
Optimizer stripped from runs/segment/train3/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/train3/weights/best.pt, 6.8MB

Validating runs/segment/train3/weights/best.pt...
Ultralytics 8.3.32 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8n-seg summary (fused): 195 layers, 3,259,234 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.81it/s]


                   all         20        158      0.909      0.942      0.949      0.703      0.919      0.921      0.933      0.642
                  eyes         19         38      0.952          1      0.994      0.649      0.965          1      0.994      0.655
                   ear         17         25      0.854      0.705      0.837      0.582      0.847      0.665      0.824      0.472
                  nose         19         19      0.924          1      0.951      0.818      0.934          1      0.951      0.749
                 mouth         19         19      0.932          1      0.972      0.782      0.944          1      0.972      0.768
                  brow         19         38      0.903          1      0.978      0.674      0.948      0.968      0.972      0.577
                  neck         19         19      0.886      0.947      0.962      0.711      0.877      0.895      0.885      0.631
Speed: 0.2ms preprocess, 3.1ms inference, 0.0ms loss, 2.0ms postproce

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7b80378b72e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004

In [10]:
import shutil

shutil.copytree('/content/runs', '/content/drive/MyDrive/Face_similarity/Results')

'/content/drive/MyDrive/Face_similarity/Results'

# Testing

In [26]:
import cv2
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow

model_path = "/content/drive/MyDrive/Face_similarity/Results/segment/train3/weights/best.pt"

image = "/content/drive/MyDrive/Face_similarity/Unseen_image/218.jpg"
cv2_image = cv2.imread(image)

h, w, _ = cv2_image.shape

model = YOLO(model_path)
results = model(image, imgsz=640, conf=0.5)

for result in results:
  print(result)
  for box in result.boxes.data.tolist():

    # for j, mask in enumerate(result.masks.data):
    #     mask = mask.cpu().numpy() * 255
    #     class_id = result.names[mask.argmax()]
    #     mask = cv2.resize(mask, (w, h))
    #     cv2.imwrite(f"mask_{j}.png", mask)
    #     cv2_imshow(mask)
    #     cv2.waitKey(0)


image 1/1 /content/drive/MyDrive/Face_similarity/Unseen_image/218.jpg: 640x640 2 eyess, 1 ear, 1 nose, 1 mouth, 2 brows, 1 neck, 15.1ms
Speed: 2.9ms preprocess, 15.1ms inference, 5.3ms postprocess per image at shape (1, 3, 640, 640)
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'eyes', 1: 'ear', 2: 'nose', 3: 'mouth', 4: 'brow', 5: 'neck'}
obb: None
orig_img: array([[[ 52,  65,  79],
        [ 54,  67,  81],
        [ 57,  70,  84],
        ...,
        [125, 141, 157],
        [126, 142, 158],
        [127, 143, 159]],

       [[ 52,  65,  79],
        [ 54,  67,  81],
        [ 57,  70,  84],
        ...,
        [123, 139, 155],
        [123, 139, 155],
        [123, 139, 155]],

       [[ 52,  65,  79],
        [ 54,  67,  81],
        [ 57,  70,  84],
        ...,
        [123, 139, 155],
        [124, 140, 156],
        [124, 140, 156]],

       